#### Introdução à Base de Dados do Airbnb

O conjunto de dados "Inside Airbnb", disponível no website "http://insideairbnb.com/", é uma valiosa fonte de informações sobre listagens de hospedagem, avaliações de hóspedes e disponibilidade de calendário em várias cidades ao redor do mundo, incluindo o Rio de Janeiro. Antes de prosseguirmos com a engenharia de dados, é importante entender os principais componentes deste conjunto de dados:

1. **Listing (Listagem):** Este conjunto de dados contém informações detalhadas sobre as propriedades listadas no Airbnb. Cada registro representa uma listagem individual e inclui informações como o tipo de propriedade, preço, localização, número de quartos, comodidades oferecidas e muito mais.

2. **Reviews (Avaliações):** O conjunto de dados de avaliações contém informações sobre as avaliações feitas por hóspedes que ficaram nas propriedades listadas. Ele inclui dados como a data da avaliação, o identificador da propriedade, os comentários escritos pelos hóspedes, e outras informações. 

3. **Calendar (Calendário):** Este conjunto de dados contém informações sobre a disponibilidade das propriedades ao longo do tempo. Ele lista as datas em que as propriedades estão disponíveis para reserva, bem como os preços para cada data.

O dicionário dos dados também está disponível no website: "http://insideairbnb.com/".

#### Passos do Projeto

1. **Aquisição de Dados e Armazenamento de Dados em PostgreSQL - Camada Bronze**
   - Baixe o conjunto de dados "Inside Airbnb" do Rio de Janeiro da fonte oficial (http://insideairbnb.com/) e promova uma estruturação simples nos dados.
   - Crie um banco de dados PostgreSQL para armazenar os dados brutos das 3 tabelas ("Listing", "Reviews" e Calendar") na camada "bronze".

<br>

2. **Data Clean - Camada Silver:**
   - Identifique e lide com valores ausentes, duplicatas e outliers nos dados brutos da camada "bronze".
   - Padronize e limpe os nomes das colunas, convertendo-os em um formato consistente.
   - Realize uma limpeza textual em campos, como descrições de propriedades, removendo caracteres especiais e erros de digitação.

<br>

3. **Data Quality - Camada Silver:**
   - Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".
   - Implemente verificações para garantir que os dados da camada "silver" estejam em conformidade com essas métricas.
   - Estabeleça um sistema de monitoramento contínuo da qualidade dos dados da camada "silver".

<br>

4. **Testes de Qualidade - Camada Silver:**
   - Utilize a biblioteca Great Expectations para criar testes de qualidade automatizados que verifiquem as expectativas definidas para os dados da camada "silver".
   - Desenvolva testes que assegurem que os dados da camada "silver" atendam às regras de negócios e aos requisitos de qualidade.

<br>

5. **Transformação de Dados com dbt - Camada Silver:**
   - Utilize a ferramenta dbt para criar a camada "silver" de dados, realizando transformações e preparando os dados da camada em questão.
   - Mantenha um controle de versão dos modelos dbt relacionados à camada "silver" e automatize a execução das transformações.

<br>

6. **Armazenamento de Dados em PostgreSQL - Camada Silver:**
   - Armazene os dados da camada "silver" no mesmo banco de dados PostgreSQL.
   - Estabeleça conexões entre o dbt e o PostgreSQL para carregar os dados transformados da camada "silver" no banco.

<br>

7. **Validação de Expectativas com Great Expectations - Camada Silver:**
   - Implemente validações adicionais usando Great Expectations nas camadas de dados da camada "silver".
   - Monitore a qualidade dos dados da camada "silver" após cada transformação e ajuste os testes de acordo.

<br>

8. **Transformação de Dados com dbt - Camada Gold:**
   - Utilize o dbt para criar a camada "gold" de dados, aplicando agregações especializadas, como médias de preços por propriedade, por período, e outras agregações especializadas.
   - Mantenha um controle de versão dos modelos dbt relacionados à camada "gold" e automatize a execução das transformações.
   - Armazene os dados da camada "gold" no mesmo banco de dados PostgreSQL, mantendo a estrutura de dados otimizada para consultas analíticas.

<br>

 9. **Apresentação e Discussão:**
    - Apresente os resultados do projeto para a turma, enfatizando os aspectos de engenharia de dados, qualidade de dados e uso de ferramentas como dbt, Great Expectations e o armazenamento em um banco de dados PostgreSQL nas camadas "bronze", "silver" e "gold".


In [15]:
# Importando as bibliotecas
import requests
import io
import pandas as pd
import sqlalchemy as sqlal
from sqlalchemy import create_engine, text as sql_text
from ydata_profiling import ProfileReport

In [16]:
# Definindo as URLs para baixar os dados
url1 = "http://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-09-22/data/listings.csv.gz"
url2 = "http://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-09-22/data/calendar.csv.gz"
url3 = "http://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-09-22/data/reviews.csv.gz"

# Função para baixar e carregar o arquivo em um dataframe
def download_df(url):
    response = requests.get(url)
    if response.status_code == 200:
        file_buffer = io.BytesIO(response.content)
        df = pd.read_csv(file_buffer, compression='gzip')
        return df
    else:
        print(f"Falha ao baixar o arquivo de {url}")
        return None

In [17]:
# Baixando e carregando os três arquivos
listings = download_df(url1)
calendar = download_df(url2)
reviews = download_df(url3)

In [18]:
listings.head(2)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,231497,https://www.airbnb.com/rooms/231497,20230922043705,2023-09-22,city scrape,Rental unit in Rio de Janeiro · ★4.73 · 1 bedr...,"This is a big studio at the end of Copacabana,...",NaN,https://a0.muscache.com/pictures/3582382/ee8ac...,1207700,...,4.92,4.90,4.65,NaN,f,4,4,0,0,0.54
1,231516,https://www.airbnb.com/rooms/231516,20230922043705,2023-09-22,city scrape,Rental unit in Rio de Janeiro · ★4.71 · 1 bedr...,"Special location of the building, on Copacaban...",NaN,https://a0.muscache.com/pictures/3671683/d74b4...,1207700,...,4.86,4.93,4.38,NaN,f,4,4,0,0,0.20


In [19]:
calendar.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,17878,2023-09-23,f,$265.00,$265.00,5.0,28.0
1,17878,2023-09-24,t,$265.00,$265.00,5.0,28.0
2,17878,2023-09-25,f,$290.00,$290.00,5.0,28.0
3,17878,2023-09-26,f,$290.00,$290.00,5.0,28.0
4,17878,2023-09-27,f,$290.00,$290.00,5.0,28.0


In [20]:
reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,231497,726625,2011-11-17,1356783,Jenny,Es un apartastudio muy bien ubicado. Tiene lo ...
1,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...
2,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...
3,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...
4,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r<b...
...,...,...,...,...,...,...
637302,980284448065462717,984807416703697120,2023-09-20,7754275,Nancy,Pour passer un séjour authentique dans un quar...
637303,979157528511400272,980387014240041411,2023-09-14,46564596,Renata,O apartamento é maravilhoso! Fiquei apaixonada...
637304,979157528511400272,983311500184670764,2023-09-18,240217909,Jorge,O loft é muito aconchegante e bem localizado. ...
637305,979548196812644568,983385320310636213,2023-09-18,537025548,Darian,Very great will stay again!


In [21]:
%run ./create_engine.ipynb

In [11]:
# Enviando os dados para o banco de dados
listings.to_sql('airbnb_listings', engine, if_exists='replace', index=False)
calendar.to_sql('airbnb_calendar', engine, if_exists='replace', index=False)
reviews.to_sql('airbnb_reviews', engine, if_exists='replace', index=False)

307

In [8]:
# Filtrando somente dados de 2023 no dataframe 'calendar'
calendar['date'] = pd.to_datetime(calendar['date'])
calendar_filtered = calendar[calendar['date'].dt.year == 2023]
calendar_filtered

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,17878,2023-09-23,f,$265.00,$265.00,5.0,28.0
1,17878,2023-09-24,t,$265.00,$265.00,5.0,28.0
2,17878,2023-09-25,f,$290.00,$290.00,5.0,28.0
3,17878,2023-09-26,f,$290.00,$290.00,5.0,28.0
4,17878,2023-09-27,f,$290.00,$290.00,5.0,28.0
...,...,...,...,...,...,...,...
11666706,985555107088259155,2023-12-27,f,$480.00,$480.00,10.0,90.0
11666707,985555107088259155,2023-12-28,f,$480.00,$480.00,10.0,90.0
11666708,985555107088259155,2023-12-29,f,$480.00,$480.00,10.0,90.0
11666709,985555107088259155,2023-12-30,f,$480.00,$480.00,10.0,90.0


In [9]:
# Renomeando a coluna 'id' do dataframe reviews e filtrando somente dados de 2023

reviews = reviews.rename(columns={'id': 'reviews_id'})
reviews['date'] = pd.to_datetime(reviews['date'])
reviews_filtered = reviews[reviews['date'].dt.year == 2023]
reviews_filtered

,listing_id,reviews_id,date,reviewer_id,reviewer_name,comments
136,231497,833997686057917406,2023-02-24,15606923,Amandine,"L'appartement était très bien situé, extrêmeme..."
137,231497,851408280047593244,2023-03-20,21755414,Emil,"thank you for hosting me, everything was perfe..."
138,231497,876844144079970089,2023-04-24,100007407,Tatiana De Lima,"Ótima localização, chuveiro gostoso e bia comu..."
139,231497,882535724244963125,2023-05-02,170818373,Luiza,Maria Luiza foi muito solícita conosco. A loca...
140,231497,901412872825027565,2023-05-28,369171341,Bruna Luiza,"Sem reclamações, foi ótimo!"
...,...,...,...,...,...,...
637302,980284448065462717,984807416703697120,2023-09-20,7754275,Nancy,Pour passer un séjour authentique dans un quar...
637303,979157528511400272,980387014240041411,2023-09-14,46564596,Renata,O apartamento é maravilhoso! Fiquei apaixonada...
637304,979157528511400272,983311500184670764,2023-09-18,240217909,Jorge,O loft é muito aconchegante e bem localizado. ...
637305,979548196812644568,983385320310636213,2023-09-18,537025548,Darian,Very great will stay again!


In [10]:
# Unindo os dataframes listings e calendar
df_merged = listings.merge(calendar_filtered, left_on='id', right_on='listing_id', how='inner')

In [12]:
# Unindo os dataframes merged (listings + calendar) e reviews
df_bronze = df_merged.merge(reviews_filtered, left_on='id', right_on='listing_id', how='inner')

In [13]:
df_bronze

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,price_y,adjusted_price,minimum_nights_y,maximum_nights_y,listing_id_y,reviews_id,date_y,reviewer_id,reviewer_name,comments
0,231497,https://www.airbnb.com/rooms/231497,20230922043705,2023-09-22,city scrape,Rental unit in Rio de Janeiro · ★4.73 · 1 bedr...,"This is a big studio at the end of Copacabana,...",NaN,https://a0.muscache.com/pictures/3582382/ee8ac...,1207700,...,$180.00,$180.00,3.0,89.0,231497,833997686057917406,2023-02-24,15606923,Amandine,"L'appartement était très bien situé, extrêmeme..."
1,231497,https://www.airbnb.com/rooms/231497,20230922043705,2023-09-22,city scrape,Rental unit in Rio de Janeiro · ★4.73 · 1 bedr...,"This is a big studio at the end of Copacabana,...",NaN,https://a0.muscache.com/pictures/3582382/ee8ac...,1207700,...,$180.00,$180.00,3.0,89.0,231497,851408280047593244,2023-03-20,21755414,Emil,"thank you for hosting me, everything was perfe..."
2,231497,https://www.airbnb.com/rooms/231497,20230922043705,2023-09-22,city scrape,Rental unit in Rio de Janeiro · ★4.73 · 1 bedr...,"This is a big studio at the end of Copacabana,...",NaN,https://a0.muscache.com/pictures/3582382/ee8ac...,1207700,...,$180.00,$180.00,3.0,89.0,231497,876844144079970089,2023-04-24,100007407,Tatiana De Lima,"Ótima localização, chuveiro gostoso e bia comu..."
3,231497,https://www.airbnb.com/rooms/231497,20230922043705,2023-09-22,city scrape,Rental unit in Rio de Janeiro · ★4.73 · 1 bedr...,"This is a big studio at the end of Copacabana,...",NaN,https://a0.muscache.com/pictures/3582382/ee8ac...,1207700,...,$180.00,$180.00,3.0,89.0,231497,882535724244963125,2023-05-02,170818373,Luiza,Maria Luiza foi muito solícita conosco. A loca...
4,231497,https://www.airbnb.com/rooms/231497,20230922043705,2023-09-22,city scrape,Rental unit in Rio de Janeiro · ★4.73 · 1 bedr...,"This is a big studio at the end of Copacabana,...",NaN,https://a0.muscache.com/pictures/3582382/ee8ac...,1207700,...,$180.00,$180.00,3.0,89.0,231497,901412872825027565,2023-05-28,369171341,Bruna Luiza,"Sem reclamações, foi ótimo!"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17860463,982590244333846157,https://www.airbnb.com/rooms/982590244333846157,20230922043705,2023-09-23,city scrape,Rental unit in Rio de Janeiro · ★New · 1 bedro...,Escape to our luxury brand new apartment on Av...,Copacabana is a vibrant beachfront neighborhoo...,https://a0.muscache.com/pictures/miso/Hosting-...,6038816,...,"$1,712.00","$1,712.00",7.0,365.0,982590244333846157,984070989742768416,2023-09-19,252364341,Maria Celina,"Location, location, location! <br/>I had previ..."
17860464,982590244333846157,https://www.airbnb.com/rooms/982590244333846157,20230922043705,2023-09-23,city scrape,Rental unit in Rio de Janeiro · ★New · 1 bedro...,Escape to our luxury brand new apartment on Av...,Copacabana is a vibrant beachfront neighborhoo...,https://a0.muscache.com/pictures/miso/Hosting-...,6038816,...,"$2,046.00","$2,046.00",7.0,365.0,982590244333846157,984070989742768416,2023-09-19,252364341,Maria Celina,"Location, location, location! <br/>I had previ..."
17860465,982590244333846157,https://www.airbnb.com/rooms/982590244333846157,20230922043705,2023-09-23,city scrape,Rental unit in Rio de Janeiro · ★New · 1 bedro...,Escape to our luxury brand new apartment on Av...,Copacabana is a vibrant beachfront neighborhoo...,https://a0.muscache.com/pictures/miso/Hosting-...,6038816,...,"$2,271.00","$2,271.00",7.0,365.0,982590244333846157,984070989742768416,2023-09-19,252364341,Maria Celina,"Location, location, location! <br/>I had previ..."
17860466,982590244333846157,https://www.airbnb.com/rooms/982590244333846157,20230922043705,2023-09-23,city scrape,Rental unit in Rio de Janeiro · ★New · 1 bedro...,Escape to our luxury brand new apartment on Av...,Copacabana is a vibrant beachfront neighborhoo...,https://a0.muscache.com/pictures/miso/Hosting-...,6038816,...,"$2,1

In [14]:
# Enviando a camada bronze para o banco de dados
df_bronze.to_sql('airbnb_bronze', engine, if_exists='replace', index=False)

# ----------------------------------------------------------------------

In [22]:
# Realizando o merge dos dataframes dentro do banco de dados
query = """
SELECT *
FROM airbnb_listings
LEFT JOIN airbnb_calendar ON airbnb_listings.id = airbnb_calendar.listing_id;
"""
#LEFT JOIN airbnb_reviews ON airbnb_listings.id = airbnb_reviews.listing_id;

# Executando a consulta para obter o dataframe resultante do merge
df_merge = pd.read_sql_query(sql=sql_text(query), con=engine.connect())

In [ ]:
profile = ProfileReport(df_bronze, title="Pandas Profiling Report") #cria o relatório
profile
#profile.to_file("resultados.html") #salva os resultados em um arquiv

In [ ]:
# Prepara os tipos de cada coluna e depois armazena os resultados no banco na camada bronze

dict_dtype={'ID': sqlal.String(),
            'Customer_ID': sqlal.String(),
            'Month': sqlal.String(),
            'Name': sqlal.String(),
            'Age': sqlal.String(),
            'SSN': sqlal.String(),
            'Occupation': sqlal.String(),
            'Annual_Income': sqlal.String(),
            'Monthly_Inhand_Salary': sqlal.Float(),
            'Num_Bank_Accounts': sqlal.Integer(),
            'Num_Credit_Card': sqlal.Integer(),
            'Interest_Rate': sqlal.Integer(),
            'Num_of_Loan': sqlal.String(),
            'Type_of_Loan': sqlal.String(),
            'Delay_from_due_date': sqlal.Integer(),
            'Num_of_Delayed_Payment': sqlal.String(),
            'Changed_Credit_Limit': sqlal.String(),
            'Num_Credit_Inquiries': sqlal.Float(),
            'Credit_Mix': sqlal.String(),
            'Outstanding_Debt': sqlal.String(),
            'Credit_Utilization_Ratio': sqlal.Float(),   
            'Credit_History_Age': sqlal.String(),
            'Payment_of_Min_Amount': sqlal.String(),
            'Total_EMI_per_month': sqlal.Float(),
            'Amount_invested_monthly': sqlal.String(),
            'Payment_Behaviour': sqlal.String(),
            'Monthly_Balance': sqlal.String()}

In [ ]:
# Enviando camada bronze para o banco de dados
df_bronze.to_sql('credit_score_bronze', engine, if_exists='replace', index=False, dtype=dict_dtype)

In [ ]:
# Recuperando camada bronze do banco de dados

query = """
SELECT * 
FROM credit_score_bronze
"""
df_silver = pd.read_sql(sql=sql_text(query), con=engine.connect())
df_silver.head(3)